In [1]:
import os
import cv2
import random
import numpy as np
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.image import img_to_array, load_img
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from tensorflow.keras.layers import Dropout
from tensorflow.keras.callbacks import EarlyStopping

# 이미지 크기 설정
img_size = (224, 224) # 이미지 크키가 크면 메모리를 많이 차지해서 설정 필요

# 데이터 로드 및 전처리 함수
def load_and_preprocess_data(image_paths, label, brightness=True):
    data = []
    labels = []

    # 랜덤하게 100장만 선택
    selected_images = random.sample(image_paths, 1000) #1000개 이미지

    for img_path in selected_images:
        img = load_img(img_path, target_size=img_size)
        img_array = img_to_array(img)

        # 이미지 명도와 채도 높이기
        if brightness:
            img_array = cv2.convertScaleAbs(img_array, alpha=1.2, beta=30)

        img_array = img_array.astype('float32') / 255.0  # 이미지를 0과 1 사이 값으로 정규화

        data.append(img_array)
        labels.append(label)

    return np.array(data), np.array(labels)


# 카테고리 리스트
categories = ["joyful", "adventure", "tradition", "nature", "cultural", "art"]
num_classes = len(categories)

# 이미지 데이터 로드 및 전처리
all_data = []
all_labels = []

for i, category in enumerate(categories):
    category_path = os.path.join("C:\\Users\\김현\\Final_Project\\crawled_img", category) # crawled_img/안에 있는 카테고리 리스트
    category_images = [os.path.join(category_path, img) for img in os.listdir(category_path)]
    
    data, labels = load_and_preprocess_data(category_images, label=i)
    all_data.extend(data)
    all_labels.extend(labels)

# 데이터 합치기 및 레이블 변환
X = np.array(all_data)
y = to_categorical(np.array(all_labels), num_classes=num_classes)

# 학습 및 테스트 데이터로 분할
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# CNN 모델 생성
model = Sequential()
model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(224, 224, 3)))
model.add(MaxPooling2D((2, 2)))
model.add(Dropout(0.25))  # Dropout 추가
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))
model.add(Dropout(0.25))  # Dropout 추가
model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))
model.add(Dropout(0.25))  # Dropout 추가
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))  # Dropout 추가
model.add(Dense(num_classes, activation='softmax'))

# 모델 컴파일
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# EarlyStopping 설정
early_stopping = EarlyStopping(monitor='val_loss', patience=2)

# 모델 학습
model.fit(X_train, y_train, epochs=50, batch_size=32, validation_data=(X_test, y_test), callbacks=[early_stopping])

Epoch 1/50
150/150 [==============================] - 318s 2s/step - loss: 1.8187 - accuracy: 0.2706 - val_loss: 1.5064 - val_accuracy: 0.4008
Epoch 2/50
150/150 [==============================] - 299s 2s/step - loss: 1.4968 - accuracy: 0.4077 - val_loss: 1.3587 - val_accuracy: 0.4742
Epoch 3/50
150/150 [==============================] - 352s 2s/step - loss: 1.3109 - accuracy: 0.4963 - val_loss: 1.2129 - val_accuracy: 0.5642
Epoch 4/50
150/150 [==============================] - 383s 3s/step - loss: 1.1568 - accuracy: 0.5633 - val_loss: 1.1088 - val_accuracy: 0.5833
Epoch 5/50
150/150 [==============================] - 435s 3s/step - loss: 1.0477 - accuracy: 0.6069 - val_loss: 1.0961 - val_accuracy: 0.6017
Epoch 6/50
150/150 [==============================] - 378s 3s/step - loss: 0.9204 - accuracy: 0.6565 - val_loss: 0.9686 - val_accuracy: 0.6617
Epoch 7/50
150/150 [==============================] - 356s 2s/step - loss: 0.7966 - accuracy: 0.7031 - val_loss: 0.9475 - val_accuracy: 0.6692

In [2]:
# 테스트 세트에서 모델 평가
evaluation_results = model.evaluate(X_test, y_test)

# 평가 결과 출력
print(f'테스트 손실: {evaluation_results[0]:.4f}')
print(f'테스트 정확도: {evaluation_results[1]*100:.2f}%')


38/38 [==============================] - 21s 550ms/step - loss: 1.1011 - accuracy: 0.6717
테스트 손실: 1.1011
테스트 정확도: 67.17%
